In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [3]:
price = pd.read_csv('./nav.csv')
portfolio = pd.read_csv('./portfolio_description.csv')

In [4]:
price.head()

,Date,VUG,QQQ,VIG,IWN,IWO,SCHG,IJS,IJT,PNQI,...,AIGE,UGA,RJN,GLD,SLV,PPLT,PALL,9081,DBB,ALUM
0,2011-02-10,64.33,58.05,54.40,73.52,91.16,31.81,74.40,75.30,38.66,...,13.43,43.42,6.33,133.18,29.51,181.70,81.66,1990,24.76,6.52
1,2011-02-11,64.75,58.42,54.78,73.13,90.66,31.73,74.04,74.96,38.71,...,13.46,42.45,6.34,132.03,29.10,181.79,81.85,1990,25.18,6.52
2,2011-02-14,64.91,58.57,54.85,73.25,91.30,31.77,74.18,75.48,38.67,...,13.61,42.37,6.39,133.08,29.30,182.98,83.94,1975,24.92,6.52
3,2011-02-15,64.69,58.50,54.72,74.15,92.31,31.96,75.00,76.24,38.41,...,13.81,43.27,6.44,133.17,29.45,181.56,83.44,1975,24.78,6.58
4,2011-02-16,65.12,58.89,55.03,74.37,92.87,32.08,75.20,76.78,36.97,...,13.75,42.71,6.41,133.93,30.00,182.18,83.34,1995,24.86,6.47


In [5]:
index = (portfolio.Mid == 'Emerging')
portfolio_1 = portfolio[index]['Ticker'].head(5)
emerging_ticker = portfolio_1.values

In [6]:
final_data = pd.concat([price.loc[:,'Date'],price.loc[:,emerging_ticker]], axis =1)
final_data.head(10)

,Date,EWZ,105010,DGS,ILF,EPI
0,2011-02-10,71.75,5782.86,52.46,51.76,22.63
1,2011-02-11,70.24,5697.47,52.03,50.61,22.12
2,2011-02-14,70.16,5713.55,51.29,50.42,21.89
3,2011-02-15,71.66,5707.09,50.94,51.19,22.23
4,2011-02-16,72.41,5709.79,51.27,51.46,22.90
5,2011-02-17,72.50,5780.17,51.30,51.70,23.05
6,2011-02-18,73.77,5791.84,51.57,52.15,23.04
7,2011-02-22,74.18,5793.35,51.59,52.43,23.35
8,2011-02-23,74.38,5784.21,52.04,52.72,23.01
9,2011-02-24,72.42,5752.09,50.86,51.28,23.09


In [8]:
ticker_names = final_data.columns
ticker_names

Index(['Date', 'EWZ', '105010', 'DGS', 'ILF', 'EPI'], dtype='object')

In [9]:
# 독립변수 만들기(t, t^2)

window_length = 252

t1 = np.arange(1, window_length+1)
t2 = np.arange(1, window_length+1)**2
X = np.array([t1, t2]).T

In [12]:
beta = {'Date': [], 'EWZ': [], '105010': [], 'DGS': [], 'ILF': [], 'EPI': []}
gamma = {'Date': [], 'EWZ': [], '105010': [], 'DGS': [], 'ILF': [], 'EPI': []}

for day in range(len(final_data)):
        if day > 252:  
            beta['Date'].append(final_data.loc[day,'Date'])
            gamma['Date'].append(final_data.loc[day,'Date'])

for name in ticker_names:
    for day in range(len(final_data)):
        if day > 252:                   
            y = final_data.loc[day-window_length+1:day, name]

            if np.all(np.isfinite(y)):
                regressor = LinearRegression()
                regressor.fit(X,y)

                beta[name].append(regressor.coef_[0])
                gamma[name].append(regressor.coef_[1])

            else:
                beta[name].append(np.nan)
                gamma[name].append(np.nan)

In [ ]:
        for i in range(n_stocks):
           
            y = dependent[:, i]

            if np.all(np.isfinite(y)):
                # create a LinearRegression object
                regressor = LinearRegression()
                
                # TODO: fit the regressor on X and y
                regressor.fit(X, y)
                
                # store the beta coefficient
                out.beta[i] = regressor.coef_[0]
                
                #TODO: store the gamma coefficient
                out.gamma[i] = regressor.coef_[1]
            else:
                # store beta as not-a-number
                out.beta[i] = np.nan
                
                # TODO: store gammas not-a-number
                out.gamma[i] = np.nan


In [9]:
beta_df = pd.DataFrame(beta).set_index('Date')
beta_df = beta_df.rank(axis = 1)
beta_df = beta_df.rename(index={})

beta_df

,EWZ,105010,DGS,ILF,EPI
Date,,,,,
2012-02-13,2.0,1.0,4.0,3.0,5.0
2012-02-14,2.0,1.0,4.0,3.0,5.0
2012-02-15,2.0,1.0,4.0,3.0,5.0
2012-02-16,2.0,1.0,4.0,3.0,5.0
2012-02-17,2.0,1.0,4.0,3.0,5.0
...,...,...,...,...,...
2019-10-02,3.0,5.0,4.0,1.0,2.0
2019-10-03,2.0,5.0,4.0,1.0,3.0
2019-10-04,2.0,5.0,4.0,1.0,3.0


In [10]:
gamma_df = pd.DataFrame(gamma).set_index('Date')
gamma_df = gamma_df.rank(axis =1)

gamma_df

,EWZ,105010,DGS,ILF,EPI
Date,,,,,
2012-02-13,4.0,5.0,2.0,3.0,1.0
2012-02-14,4.0,5.0,2.0,3.0,1.0
2012-02-15,4.0,5.0,2.0,3.0,1.0
2012-02-16,4.0,5.0,2.0,3.0,1.0
2012-02-17,4.0,5.0,2.0,3.0,1.0
...,...,...,...,...,...
2019-10-02,4.0,1.0,2.0,5.0,3.0
2019-10-03,4.0,1.0,2.0,5.0,3.0
2019-10-04,4.0,1.0,2.0,5.0,3.0


In [11]:
conditional_factor = (beta_df * gamma_df).rank(axis = 1)
conditional_factor

,EWZ,105010,DGS,ILF,EPI
Date,,,,,
2012-02-13,3.5,1.5,3.5,5.0,1.5
2012-02-14,3.5,1.5,3.5,5.0,1.5
2012-02-15,3.5,1.5,3.5,5.0,1.5
2012-02-16,3.5,1.5,3.5,5.0,1.5
2012-02-17,3.5,1.5,3.5,5.0,1.5
...,...,...,...,...,...
2019-10-02,5.0,1.5,4.0,1.5,3.0
2019-10-03,3.5,1.5,3.5,1.5,5.0
2019-10-04,3.5,1.5,3.5,1.5,5.0


In [12]:
conditional_factor.to_csv('alpha_factor_emerging.csv', mode = 'a')

In [15]:
index = portfolio.Mid == 'Metal'
portfolio_1 = portfolio[index]['Ticker']
metal_ticker = portfolio_1.values

In [17]:
final_data = pd.concat([price.loc[:,'Date'],price.loc[:,metal_ticker]], axis =1)
final_data.head(10)

,Date,GLD,SLV,PPLT,PALL,9081,DBB,ALUM
0,2011-02-10,133.18,29.51,181.70,81.66,1990,24.76,6.52
1,2011-02-11,132.03,29.10,181.79,81.85,1990,25.18,6.52
2,2011-02-14,133.08,29.30,182.98,83.94,1975,24.92,6.52
3,2011-02-15,133.17,29.45,181.56,83.44,1975,24.78,6.58
4,2011-02-16,133.93,30.00,182.18,83.34,1995,24.86,6.47
5,2011-02-17,133.78,30.05,182.37,84.13,1995,25.22,6.20
6,2011-02-18,134.53,29.89,180.87,83.04,2005,24.70,6.13
7,2011-02-22,134.97,31.19,178.19,80.35,2000,24.57,6.13
8,2011-02-23,136.67,32.12,176.00,76.48,2005,24.59,6.23
9,2011-02-24,137.48,32.51,177.88,77.97,2005,24.96,6.13


In [18]:
ticker_names = ['GLD', 'SLV', 'PPLT', 'PALL', '9081', 'DBB', 'ALUM']

In [19]:
window_length = 252

t1 = np.arange(1, window_length+1)
t2 = np.arange(1, window_length+1)**2
X = np.array([t1, t2]).T

In [15]:
beta = {'Date': [], 'GLD': [], 'SLV': [], 'PPLT': [], 'PALL':[], '9081':[], 'DBB':[], 'ALUM':[]}
gamma = {'Date': [], 'GLD': [], 'SLV': [], 'PPLT': [], 'PALL':[], '9081':[], 'DBB':[], 'ALUM':[]}

for day in range(len(final_data)):
        if day > 252:  
            beta['Date'].append(final_data.loc[day,'Date'])
            gamma['Date'].append(final_data.loc[day,'Date'])

for name in ticker_names:
    for day in range(len(final_data)):
        if day > 252:                   
            y = final_data.loc[day-window_length+1:day, name]

            if np.all(np.isfinite(y)):
                regressor = LinearRegression()
                regressor.fit(X,y)

                beta[name].append(regressor.coef_[0])
                gamma[name].append(regressor.coef_[1])

            else:
                beta[name].append(np.nan)
                gamma[name].append(np.nan)

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [21]:
beta_df = pd.DataFrame(beta).set_index('Date')
beta_df = beta_df.rank(axis = 1)
beta_df = beta_df.rename(index={})

beta_df

,GLD,SLV,PPLT,PALL,9081,DBB,ALUM
Date,,,,,,,
2012-02-13,7.0,5.0,4.0,1.0,6.0,2.0,3.0
2012-02-14,7.0,5.0,4.0,1.0,6.0,2.0,3.0
2012-02-15,7.0,5.0,4.0,1.0,6.0,2.0,3.0
2012-02-16,7.0,5.0,3.0,1.0,6.0,2.0,4.0
2012-02-17,7.0,5.0,3.0,1.0,6.0,2.0,4.0
...,...,...,...,...,...,...,...
2019-10-02,2.0,3.0,1.0,6.0,7.0,5.0,4.0
2019-10-03,2.0,3.0,1.0,6.0,7.0,5.0,4.0
2019-10-04,3.0,2.0,1.0,6.0,7.0,5.0,4.0


In [22]:
gamma_df = pd.DataFrame(gamma).set_index('Date')
gamma_df = gamma_df.rank(axis =1)

gamma_df

,GLD,SLV,PPLT,PALL,9081,DBB,ALUM
Date,,,,,,,
2012-02-13,1.0,3.0,2.0,4.0,7.0,5.0,6.0
2012-02-14,1.0,3.0,2.0,4.0,7.0,5.0,6.0
2012-02-15,1.0,3.0,2.0,4.0,7.0,5.0,6.0
2012-02-16,1.0,3.0,2.0,4.0,7.0,5.0,6.0
2012-02-17,1.0,3.0,2.0,4.0,7.0,5.0,6.0
...,...,...,...,...,...,...,...
2019-10-02,7.0,5.0,6.0,2.0,1.0,3.0,4.0
2019-10-03,7.0,5.0,6.0,2.0,1.0,3.0,4.0
2019-10-04,7.0,5.0,6.0,2.0,1.0,3.0,4.0


In [23]:
conditional_factor = (beta_df * gamma_df).rank(axis = 1)
conditional_factor

,GLD,SLV,PPLT,PALL,9081,DBB,ALUM
Date,,,,,,,
2012-02-13,2.0,5.0,3.0,1.0,7.0,4.0,6.0
2012-02-14,2.0,5.0,3.0,1.0,7.0,4.0,6.0
2012-02-15,2.0,5.0,3.0,1.0,7.0,4.0,6.0
2012-02-16,3.0,5.0,2.0,1.0,7.0,4.0,6.0
2012-02-17,3.0,5.0,2.0,1.0,7.0,4.0,6.0
...,...,...,...,...,...,...,...
2019-10-02,4.0,5.5,1.0,3.0,2.0,5.5,7.0
2019-10-03,4.0,5.5,1.0,3.0,2.0,5.5,7.0
2019-10-04,7.0,3.0,1.0,4.0,2.0,5.0,6.0


In [24]:
conditional_factor.to_csv('alpha_factor_metal.csv', mode = 'a')